Code for Simulator - updates Geojson file with Completion status and time

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import random
from datetime import datetime, timedelta, timezone
import os
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
from tatc.schemas import TwoLineElements
from tatc.schemas import Point
from tatc.analysis import collect_observations
from tatc import utils
from tatc.schemas import Instrument
from tatc.schemas import WalkerConstellation, SunSynchronousOrbit
from tatc.utils import (
    swath_width_to_field_of_regard,
    along_track_distance_to_access_time,
)
import datetime
from tatc.analysis import collect_multi_observations
from tatc.schemas import Satellite
from datetime import datetime, timedelta, timezone
from tatc.schemas import Point
import logging
logging.basicConfig(level=logging.INFO)
from tatc.analysis import collect_ground_track
from tatc.analysis import compute_ground_track
from tatc.schemas import PointedInstrument, WalkerConstellation, SunSynchronousOrbit
from tatc.utils import swath_width_to_field_of_regard, swath_width_to_field_of_view
import pytz

class Simulator():
    def __init__(self):
        pass

    # This function uses the tatc library to define the  Snow_globe Constellation, we define tle, instrument 
    
    def const(self):
        roll_angle = (30 + 33.5)/2
        roll_range = (33.5 - 30)
        start = datetime(2019, 1, 1, tzinfo=timezone.utc)
        self.constellation = WalkerConstellation(
            name="SnowGlobe Ku",
            orbit=SunSynchronousOrbit(
                altitude=555e3, 
                equator_crossing_time="06:00:30", 
                equator_crossing_ascending=False,
                epoch=start
            ),
            number_planes=1,
            number_satellites=5,
            instruments=[
                PointedInstrument(
                    name="SnowGlobe Ku-SAR",
                    roll_angle=-roll_angle,
                    field_of_regard=2*roll_angle + swath_width_to_field_of_regard(555e3, 50e3),
                    along_track_field_of_view=swath_width_to_field_of_view(555e3, 50e3, 0),
                    cross_track_field_of_view=roll_range + swath_width_to_field_of_view(555e3, 50e3, roll_angle),
                    is_rectangular=True
                )
            ]
        )
        satellites = self.constellation.generate_members()
        self.satellite_dict = {sat.name: sat for sat in satellites}

    # This function reads the master geojson file and filter unprocessed requests
    # (non empty simulation status indicates, that location/request was processed) 
    # key output = self.filtered_req (pandas dataframe with filtered rows )  
    
    def user_request(self):
        self.req = gpd.read_file('Master_file')      
        self.filtered_req = self.req[self.req['simulation_status'].isna() | (self.req['simulation_status'] == "None")]         

    # This function uses tatc library multi - observations to generate the list of observation times
    # It uses all the locations from the user_request function, the simulation time at each step is given as input    
    # key output = self.combined_results, is a pandas data frame, column epoch returns the obs time (we sort it ascending)

    def opportunity(self,start_time=None):
        self.const()
        self.user_request() 
        start_time = start_time or self._time      
        end = start_time + timedelta(2)
        combined_results = pd.DataFrame()
        for index,row in self.filtered_req.iterrows():    
            loc = Point(id=row['id'],latitude=row['latitude'],longitude=row['longitude'])
            results = collect_multi_observations(loc, self.constellation, self._time, end)
            combined_results = pd.concat([combined_results, results], ignore_index=True)    
        self.combined_results = combined_results.sort_values(by='epoch', ascending=True)
        
   # whenever the planner uploades geojson file we want requests to be updated
   # (to incorporate changes from optimizer or updates from simulation , ie simulation status updates)
   
    def execute(self,init_time, duration, time_step): 
        # intilization
        self.const()
        self.user_request()
        self._time = self._next_time = self._init_time = init_time
        self._duration = duration
        self._time_step = time_step

        while self._time < self._init_time + self._duration:
            flag = 0
            print(f"Current time {self._time}")            
            self._next_time = self._time + self._time_step
            print(f"advancing to {self._next_time}") 

            # updating user requests - read filtered rows       
            self.user_request() # reading updated master file        

            # Error handler
            if self.filtered_req.empty:
                logging.info("No observations available. Skipping to next time step.")
                # Can use this condition to reset the master file
                self._time = self._next_time
                continue
            
            self.opportunity() # updating observations list
            if self.combined_results.empty:
                logging.error("combined_results is empty! No observations until next time step! Skipping to next")
                # self._time = self._next_time
                continue            

            self.rs = self.combined_results # writing the observations pandas dataframe to new variable
            self.observation_time = self.rs['epoch'].iloc[0] # latest possible observation            
            self.id = self.rs['point_id'].iloc[0] # point id for the above observation
            self.coord = self.rs['geometry'].iloc[0] # location for the observation
            self.sat = self.rs['satellite'].iloc[0] # satellite collecting the observation
            prev_observation_time = None
            # This below loop is written to handle the time step(1 day), there can be multiple observations within a day 
            # it loops through all the observations possible until the next time step
            counter = 0
            len_rs = len(self.rs)

            while self.observation_time < self._next_time:
                len_rs = len(self.rs)
                # if self.observation_time == prev_observation_time:
                #     logging.warning("No progress in observations, breaking loop.")
                #     # self._time = self._next_time
                #     break

                # Ensuring no more than 1 observation is collected at a time
                if prev_observation_time is None:
                    prev_observation_time = self.observation_time
                elif self.observation_time <= (prev_observation_time + timedelta(minutes=1)) and (counter <= len_rs):
                    counter += 1
                    self.observation_time = self.rs['epoch'].iloc[counter]
                    self.id = self.rs['point_id'].iloc[counter]                    
                    self.sat = self.rs['satellite'].iloc[counter]
                    continue
                elif counter >= len_rs:
                    break
                
                # Constellation Capacity Logic
                if (np.random.rand() <= 0.25) & (counter <= len_rs):
                    counter += 1                    
                    self.observation_time = self.rs['epoch'].iloc[counter]
                    self.id = self.rs['point_id'].iloc[counter]                    
                    self.sat = self.rs['satellite'].iloc[counter]
                elif counter >= len_rs:
                    break
                else:             
                
                    prev_observation_time = self.observation_time
                    print(f"Next observation {self.observation_time}") 
                    req = self.req # reads the requests file

                    # Formatting
                    req['completion_date'] = pd.to_datetime(req['completion_date'], errors='coerce')  # Ensure completion_date is datetime
                    req['simulation_status'] = req['simulation_status'].astype(str)  # Ensure simulation_status is string                    
                    req['satellite'] = req['satellite'].astype(str)
                    # format time as required in gejson file
                    t = self.observation_time
                    # t = self.observation_time.replace(tzinfo=None)         
                    
                    # Assigning values to master file
                    req.loc[req.id == self.id, 'completion_date'] = t
                    req.loc[req.id == self.id, 'simulation_status'] = 'Completed'
                    # req.loc[req.id == self.id, 'request_status'] = 'Completed'
                    req.loc[req.id == self.id, 'satellite'] = self.sat

                    # Groundtrack information
                    sat_object = self.satellite_dict.get(self.sat)
                    results = collect_ground_track(sat_object,[t],crs='spice')
                    req.loc[req.id == self.id, 'geometry'] = results['geometry'].iloc[0]         
                    
                    # Save the updated DataFrame back to the Master Geojson file
                    
                    req.to_file("Master_file", driver="GeoJSON")


                    # Regenerating observations with respect to updated list
                    # calling user_request and opportunity will now exclude the entries processed above(sompleted status) and generate new list
                    self.user_request()
                    counter = 0
                    flag += 1

                    # Error handler
                    if self.filtered_req.empty:
                        logging.info("No observations available. Skipping to next time step.")
                    # Can use this condition to reset the master file
                        # self._time = self._next_time
                        continue
                    
                    self.opportunity(self.observation_time) 

                    if self.combined_results.empty:
                        logging.error("combined_results is empty! No observations until next time step! Skipping to next")
                        # self._time = self._next_time
                        continue

                    self.rs = self.combined_results
                    self.observation_time = self.rs['epoch'].iloc[counter]
                    self.id = self.rs['point_id'].iloc[counter]                    
                    self.sat = self.rs['satellite'].iloc[counter]
                    len_rs = len(self.rs)
            
            # Simulation advances to next time
            # Filter data and write geojson

            if flag>0:
                self.user_request()
                # Filter data for each day(self.time)
                file_name = f"Simulator_Output_{pd.to_datetime(self._time).strftime('%Y-%m-%d')}"
                filtered_data = self.req[self.req['completion_date'].dt.date == pd.to_datetime(self._time).date()]               
                filtered_data.to_file(file_name, driver="GeoJSON")  
                flag = 0         
            self._time = self._next_time     


s = Simulator()
from datetime import datetime, timedelta, timezone

start = datetime(2019, 3, 10, tzinfo=timezone.utc)
duration = timedelta(days=5)
time_step = timedelta(days=1)
s.execute(start, duration, time_step)

Read the simulation output

In [66]:
master = gpd.read_file('Master_file')
filtered_req = master[master['simulation_status'].isna()] 
filtered_req
master.head(3)

,id,time,final_eta,Planner_geometry,centroid,latitude,longitude,expiration_date,simulation_status,completion_date,satellite,geometry
0,1,2019-03-10,0.003591,"POLYGON ((-91.94683 37.024602, -91.94683 37.63...",POINT (-92.252265 37.329427),37.329427,-92.252265,None,Completed,None,None,None


In [12]:
o_p = gpd.read_file('Simulator_Output_2019-03-13')
o_p

,id,time,final_eta,Planner_geometry,centroid,latitude,longitude,expiration_date,simulation_status,completion_date,satellite,geometry
0,13,2019-03-10,0.007196,"POLYGON ((-96.222918 38.840957, -96.222918 39....",POINT (-96.528353 39.14574),39.145740,-96.528353,None,Completed,2019-03-13 00:00:01.109000+00:00,SnowGlobe Ku 3,"POLYGON Z ((-92.71491 42.92302 0, -92.73127 42..."
1,15,2019-03-10,0.003550,"POLYGON ((-91.33596 38.840957, -91.33596 39.45...",POINT (-91.641395 39.14574),39.145740,-91.641395,None,Completed,2019-03-13 23:35:03.881000+00:00,SnowGlobe Ku 4,"POLYGON Z ((-85.94297 39.90656 0, -85.95847 39..."
2,17,2019-03-10,0.008129,"POLYGON ((-101.109876 39.295046, -101.109876 3...",POINT (-101.415311 39.599818),39.599818,-101.415311,None,Completed,2019-03-13 13:30:11.614000+00:00,SnowGlobe Ku 2,"POLYGON Z ((-109.12222 40.75634 0, -109.13765 ..."
3,20,2019-03-10,0.009825,"POLYGON ((-95.612049 39.295046, -95.612049 39....",POINT (-95.917484 39.599818),39.599818,-95.917484,None,Completed,2019-03-13 00:18:12.260000+00:00,SnowGlobe Ku 2,"POLYGON Z ((-96.63902 39.39257 0, -96.65438 39..."
4,22,2019-03-10,0.003589,"POLYGON ((-91.33596 39.295046, -91.33596 39.90...",POINT (-91.641395 39.599818),39.599818,-91.641395,None,Completed,2019-03-13 12:32:54.104000+00:00,SnowGlobe Ku 5,"POLYGON Z ((-94.86896 40.35466 0, -94.88428 40..."
5,23,2019-03-10,0.003415,"POLYGON ((-90.725091 39.295046, -90.725091 39....",POINT (-91.030526 39.599818),39.599818,-91.030526,None,Completed,2019-03-13 12:13:56.229000+00:00,SnowGlobe Ku 1,"POLYGON Z ((-90.23568 39.72587 0, -90.25084 39..."
6,24,2019-03-10,0.007357,"POLYGON ((-101.109876 39.749134, -101.109876 4...",POINT (-101.415311 40.053895),40.053895,-101.415311,None,Completed,2019-03-13 00:37:29.741000+00:00,SnowGlobe Ku 1,"POLYGON Z ((-101.56226 39.98238 0, -101.57777 ..."
7,27,2019-03-10,0.011437,"POLYGON ((-96.222918 39.749134, -96.222918 40....",POINT (-96.528353 40.053895),40.053895,-96.528353,None,Completed,2019-03-13 23:54:25.825000+00:00,SnowGlobe Ku 3,"POLYGON Z ((-90.93419 40.77349 0, -90.94992 40..."
8,33,2019-03-10,0.007294,"POLYGON ((-96.833788 40.203223, -96.833788 40....",POINT (-97.139223 40.507973),40.507973,-97.139223,None,Completed,2019-03-13 13:10:44.516000+00:00,SnowGlobe Ku 3,"POLYGON Z ((-104.04923 41.94328 0, -104.06499 ..."
9,35,2019-03-10,0.005982,"POLYGON ((-95.612049 40.203223, -95.612049 40....",POINT (-95.917484 40.507973),40.507973,-95.917484,None,Completed,2019-03-13 12:51:46.817000+00:00,SnowGlobe Ku 4,"POLYGON Z ((-99.4227 41.30398 0, -99.43828 41...."
